In [214]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import json
import scipy.stats as stats
import statsmodels.stats.multitest as smm
from statsmodels.stats.multitest import fdrcorrection

In [215]:
feature_names = json.loads(open('utils/lut_dict.txt').read())

In [216]:
# HISTOGRAM
#############################################################

# create dataframes
important_features = pd.read_csv("out/important_features.csv")
ml_dataframe = pd.read_csv("stats/ml_dataframe.csv")

# rename columns 
important_features = important_features.rename(columns={'feature': 'importance'})
important_features = important_features.rename(columns={'Unnamed: 0': 'feature'})

# print all important features into a new list
important_features_list = important_features['feature'].tolist()

# save names of features where importance >=5 to a list called top_features
top_features = important_features[important_features['importance'] >= 5]
top_features = top_features['feature'].tolist()

# make separate dataframe for top features' importance 
top_features_importance = important_features[important_features['importance'] >= 5]

# reading LUT
region_names = np.array(list([feature_names[feature] for feature in important_features['feature']]))

# add label column
important_features['feature type'] = np.where(important_features['feature'].str.contains('thickness'), "thickness", 
                                       np.where(important_features['feature'].str.contains('area'), "area", "volume"))
important_features['feature'] = region_names

# make dictionary with feature and importance of important features
feature_importance_dict = dict(zip(important_features['feature'], important_features['importance']))

print(top_features_importance)

                               feature  importance
0                               lh_PuI          10
1                               lh_VPL          10
2   lh_G_oc-temp_med-Lingual_thickness          10
3              lh_G_insular_short_area          10
4                          rh_CA3-body           8
5                           lh_fimbria           7
6         rh_G&S_transv_frontopol_area           6
7    rh_S_collat_transv_post_thickness           6
8               rh_G_front_middle_area           6
9                     lh_G_rectus_area           6
10              lh_G_temporal_inf_area           5
11       lh_G_temp_sup-Plan_tempo_area           5


In [217]:
# generate histogram
with plt.xkcd():
    ax1 = sns.catplot(data=important_features, kind="bar", x="importance", y="feature", height=15, aspect=2, hue="feature type", dodge=False, palette=['#5Ef2A8', '#6E8CE3', '#FF9F21'])
    plt.xticks([0,1,2,3,4,5,6,7,8,9,10])    
    plt.title("Feature Importance", fontsize=40)
    plt.xlabel("Importance", fontsize=30)
    plt.ylabel("Feature", fontsize=30)
    plt.savefig('out/important_features.png', dpi=300, bbox_inches='tight')
    plt.clf()

findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd S

In [218]:
# FEATURE WEIGHTS
#############################################################

# make a graph to visualize the feature weights data in outs folder ([fold]_feature_weights.csv)
#load df from all folds and merge into one dataframe

#create new dataframe for each fold file

dataframes = []  # List to store the dataframes

for i in range(0,10):
    filename=('out/{}_feature_weights.csv'.format(i))
     # Generate the filename
    df = pd.read_csv(filename)  # Read the CSV file into a dataframe
    dataframes.append(df)

# rename first column of each dataframe to fold #
for i in range(0,10):
    dataframes[i] = dataframes[i].rename(columns={'Unnamed: 0': 'fold #'})
    dataframes[i]['fold #'] = i

# merge all 10 dataframes within dataframes list into a single dataframe
all_weights = pd.concat(dataframes)
all_weights.head()

,fold #,lh_PuI,rh_S_occipital_ant_thickness,lh_G_pariet_inf-Supramar_area,lh_fimbria,lh_VPL,lh_G_oc-temp_med-Lingual_thickness,rh_G_temp_sup-Plan_tempo_area,rh_G&S_transv_frontopol_area,rh_S_collat_transv_post_thickness,...,lh_MDl,lh_Lat_Fis-ant-Vertical_area,lh_G_cingul-Post-dorsal_area,rh_Medial-nucleus,rh_VM,rh_S_orbital-H_Shaped_area,rh_CA3-head,rh_S_oc-temp_med&Lingual_area,lh_S_pericallosal_area,rh_hippocampal-fissure
0,0,0.377399,0.208406,-0.234949,0.152990,-0.498932,0.607970,0.139455,0.132272,0.366735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1,0.443621,NaN,NaN,0.318967,-0.782786,0.635613,NaN,0.187614,0.406839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2,0.258765,NaN,NaN,NaN,-0.803627,0.759131,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3,0.387491,NaN,-0.190424,NaN,-0.646881,0.715375,NaN,0.114446,0.365059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4,0.486563,NaN,-0.157687,0.216399,-0.476016,0.809729,NaN,0.184968,0.486122,...,0.206996,-0.115808,0.099105,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [219]:
# melt
all_weights = pd.melt(all_weights, id_vars=['fold #'], var_name='feature', value_name='weight')

print(all_weights)


     fold #                 feature    weight
0         0                  lh_PuI  0.377399
1         1                  lh_PuI  0.443621
2         2                  lh_PuI  0.258765
3         3                  lh_PuI  0.387491
4         4                  lh_PuI  0.486563
..      ...                     ...       ...
455       5  rh_hippocampal-fissure       NaN
456       6  rh_hippocampal-fissure       NaN
457       7  rh_hippocampal-fissure       NaN
458       8  rh_hippocampal-fissure       NaN
459       9  rh_hippocampal-fissure -0.150034

[460 rows x 3 columns]


In [220]:
# keep only top weights / replace column headers with feature_names from lut_dict.txt

# make top_weights dataframe where feature is in top_features list and sorted based on importance of that feature in important_features dataframe, higher importance comes first
top_weights = all_weights[all_weights['feature'].isin(top_features)]

# abbreviate names
region_names = np.array(list([feature_names[feature] for feature in top_weights['feature']]))
top_weights['feature'] = region_names

#do same for top_features_importance
region_names = np.array(list([feature_names[feature] for feature in top_features_importance['feature']]))
top_features_importance['feature'] = region_names


# add column for importance of that feature from important_features dataframe
top_weights['importance'] = all_weights['feature'].map(important_features.set_index('feature')['importance'])

# abs value
top_weights['weight'] = top_weights['weight'].abs()

# reorder top_weights dataframe based on importance of feature in feature_importance_dict
top_weights['importance'] = top_weights['feature'].map(feature_importance_dict)
top_weights = top_weights.sort_values(by=['importance'], ascending=False)
top_weights = top_weights.sort_values(by=['importance'], ascending=False).reset_index(drop=True)



         feature  importance
0        L PuI V          10
1        L VPL V          10
2       L LinG T          10
3     L ShoInG A          10
4   R CA3 Body V           8
5    L Fimbria V           7
6   R TrFPoG&S A           6
7   R PosTrCoS T           6
8        R MFG A           6
9         L RG A           6
10     L InfTG A           5
11       L TPl A           5


/var/folders/05/4bm5nzbd43d1zy10xs1n6j780000gn/T/ipykernel_27974/513380162.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_weights['feature'] = region_names
/var/folders/05/4bm5nzbd43d1zy10xs1n6j780000gn/T/ipykernel_27974/513380162.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_features_importance['feature'] = region_names
/var/folders/05/4bm5nzbd43d1zy10xs1n6j780000gn/T/ipykernel_27974/513380162.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [229]:
# boxplot of the dataframe where y=weight and x=feature

fig = plt.figure(figsize=(11, 5))
with plt.xkcd():
    ax = sns.boxplot(data=top_weights, x="feature", y="weight",orient="v", palette="pastel", order=top_weights.groupby('feature').mean()
                .sort_values('weight', ascending=False).index, showmeans=True, meanprops={"marker":"o","markerfacecolor":"white", 
                                                                                            "markersize":"20", "markeredgecolor":"gray", "markeredgewidth":"2"})
    # x-axis labels 
    xlabels = top_weights.groupby('feature').mean().sort_values('weight', ascending=False).index.tolist()
    top_features_importance = top_features_importance.set_index('feature').loc[xlabels].reset_index()
    
    # store means of boxplot in a list
    means = []
    for i, _ in  enumerate(top_features):
        means.append((top_weights.groupby('feature').mean().sort_values('weight', ascending=False)['weight'][i]))

    plt.xlabel('Feature')
    plt.ylabel('Weight (abs. value)')
    plt.title('Top Feature Weights', size=20)
    plt.xticks(rotation=30)
    sns.stripplot(data=top_weights, x="feature", y="weight", color="gray", zorder=1, size=4, jitter=0.2, order=top_weights.groupby('feature')
                .mean().sort_values('weight', ascending=False).index)

    # add text labels corresponding to top_features_importance at each of the means
    for i,_ in enumerate(top_features):
        plt.text(x=i, y=(means[i]-0.005), s=top_features_importance['importance'][i], color='gray', ha="center", fontsize=12)

    plt.savefig('out/feature_weights.png', dpi=300, bbox_inches='tight')
    plt.clf()


findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd Script' not found.
findfont: Font family 'Comic Neue' not found.
findfont: Font family 'xkcd' not found.
findfont: Font family 'xkcd S

In [ ]:
# T-TEST
#############################################################

# create dataframe with only top features
top_features_dataframe = ml_dataframe[top_features]
top_features_dataframe['diagnosis'] = ml_dataframe['diagnosis']

# calculate mean for each column and put in list
means = []
for i, _ in enumerate(top_features):
    means.append(top_features_dataframe.mean()[top_features[i]])

# melt and organize dataframe for top features
top_features_dataframe = top_features_dataframe.melt(id_vars=['diagnosis'], var_name='feature', value_name='value').fillna(0)
top_features_dataframe['feature'] = [feature_names[feature] for feature in top_features_dataframe['feature']]

# # create and melt dataframe for all important features
# important_features_dataframe = ml_dataframe[important_features_list]
# important_features_dataframe['diagnosis'] = ml_dataframe['diagnosis']
# important_features_dataframe = important_features_dataframe.melt(id_vars=['diagnosis'], var_name='feature', value_name='value').fillna(0)
# important_features_dataframe['feature'] = [feature_names[feature] for feature in important_features_dataframe['feature']]

# set up groups for t-test
ms_dataframe = ml_dataframe[ml_dataframe['diagnosis'] == 'MS']
ms_tn_dataframe = ml_dataframe[ml_dataframe['diagnosis'] == 'MS-TN']

In [ ]:
# run t-test

p_vals = []
p_corrected = []
stats_vals = []

# running t-test on top features
for feature in top_features:
    input_ms = ms_dataframe[feature]
    input_ms_tn = ms_tn_dataframe[feature]
    ttest1 = stats.ttest_ind(input_ms, input_ms_tn, equal_var=True, nan_policy="omit", permutations=None, random_state=None, alternative="two-sided", trim=0)
    #nan_policy='propagate' when all data there
    p_vals.append(ttest1.pvalue)
    stats_vals.append(ttest1.statistic)

In [ ]:
# # running t-test on all important features
# for feature in important_features_list:
#     input_ms = ms_dataframe[feature]
#     input_ms_tn = ms_tn_dataframe[feature]
#     ttest1 = stats.ttest_ind(input_ms, input_ms_tn, equal_var=True, nan_policy="omit", permutations=None, random_state=None, alternative="two-sided", trim=0)
#     p_vals.append(ttest1.pvalue)
#     stats_vals.append(ttest1.statistic)

In [ ]:
# correct p-values for multiple tests

rejected_pval, p_corrected = fdrcorrection(p_vals, alpha=0.05, method='indep')

# replace rejected p-values with "NS"
for i in range(len(p_corrected)):
    if rejected_pval[i] == False:
        p_corrected[i] = 1
    else:
        p_corrected[i] = p_corrected[i]

# save p_corrected of top_features to a csv
p_corrected_df = pd.DataFrame(p_corrected, columns=['p_corrected'])
p_corrected_df['feature'] = top_features


# if saving all important features
# p_corrected_df['feature'] = important_features_list
# significant_pvals_dict = {}
# for pval in p_corrected_df['p_corrected']:
#     if pval != 1:
#         # add feature name and pval to dictionary
#         significant_pvals_dict[p_corrected_df['feature'][p_corrected_df['p_corrected'] == pval].values[0]] = pval
#         # sort by descending
# significant_pvals_dict = dict(sorted(significant_pvals_dict.items(), key=lambda item: item[1], reverse=False))

# print(significant_pvals_dict)
# p_corrected_df.to_csv('out/p_corrected_all_significant.csv')


# save p_corrected of top features to a csv
p_corrected_df.to_csv('out/p_corrected.csv')

# copy df into new one  
p_corrected_df_values = p_corrected_df.copy()

p_corrected_df['p_corrected'] = ['NS' if p_corrected == 1 else '*' if p_corrected > 0.01 
                                 else '**' if p_corrected > 0.001 else '***' for p_corrected in p_corrected_df['p_corrected']]
p_values_codes = p_corrected_df['p_corrected'].tolist()

In [ ]:
# plotting the values of the top predictors

with plt.xkcd():
    sns.catplot(data=top_features_dataframe, kind="box", x="feature", y="value", hue="diagnosis", height=8, aspect=2, palette="pastel")        
    plt.xticks(rotation=30)
    # add text labels corresponding to top_features_importance at each of the means
    # for i,_ in enumerate(top_features):
    #     plt.text(x=i, y=(means[i]), s=p_values_codes[i], color='gray', ha="center", fontsize=12)
    plt.yscale('log')
    plt.xlabel('Feature')
    plt.ylabel('Size')
    plt.title('Top Features', size=20)
    plt.savefig('out/top_feature_values.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.clf()

In [ ]:
# make separate dataframes depending on region
top_features_dataframe_hippo = top_features_dataframe[top_features_dataframe['feature'].str.contains('CA1|CA2|CA3|CA4|DG|SUB|ERC|Fimbria')]
top_features_dataframe_thal = top_features_dataframe[top_features_dataframe['feature'].str.contains('VPL|PuI')]
top_features_dataframe_t = top_features_dataframe[top_features_dataframe['feature'].str.endswith('T')]
top_features_dataframe_a = top_features_dataframe[top_features_dataframe['feature'].str.endswith('A')]

# print unique values of top features thal and hippo
print(top_features_dataframe_thal['feature'].unique())
print(top_features_dataframe_hippo['feature'].unique())

In [ ]:
# do different subplots for top features separated from the different regions

with plt.xkcd():
    fig, ax = plt.subplots(2, 2, figsize=(20, 20))
    plt.suptitle('Top Features by Region', size=40)

    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    sns.boxplot(ax=ax[0,0], data=top_features_dataframe_t, x="feature", y="value", hue="diagnosis", palette="pastel")
    sns.boxplot(ax=ax[0,1], data=top_features_dataframe_a, x="feature", y="value", hue="diagnosis", palette="pastel")
    sns.boxplot(ax=ax[1,0], data=top_features_dataframe_thal, x="feature", y="value", hue="diagnosis", palette="pastel")
    sns.boxplot(ax=ax[1,1], data=top_features_dataframe_hippo, x="feature", y="value", hue="diagnosis", palette="pastel")

    ax[0,0].set_ylabel('Size')
    ax[0,1].set_ylabel('Size')
    ax[1,0].set_ylabel('Size')
    ax[1,1].set_ylabel('Size')

    plt.setp(ax[0,0].get_xticklabels(), rotation=30)
    plt.setp(ax[0,1].get_xticklabels(), rotation=30)
    plt.setp(ax[1,0].get_xticklabels(), rotation=30)
    plt.setp(ax[1,1].get_xticklabels(), rotation=30)

    ax[0,0].set_title('Cortical Thickness', size=20)
    ax[0,1].set_title('Cortical Area', size=20)
    ax[1,0].set_title('Thalamic Volume', size=20)
    ax[1,1].set_title('Hippocampal Volume', size=20)

    # add scatterplot on top of boxplot
    sns.stripplot(ax=ax[0,0], data=top_features_dataframe_t, x="feature", y="value", hue="diagnosis", palette="pastel", 
                  dodge=True, size=5, edgecolor="white", linewidth=1, legend=False)   
    sns.stripplot(ax=ax[0,1], data=top_features_dataframe_a, x="feature", y="value", hue="diagnosis", palette="pastel",
                    dodge=True, size=5, edgecolor="white", linewidth=1, legend=False)
    sns.stripplot(ax=ax[1,0], data=top_features_dataframe_thal, x="feature", y="value", hue="diagnosis", palette="pastel",
                    dodge=True, size=5, edgecolor="white", linewidth=1, legend=False)
    sns.stripplot(ax=ax[1,1], data=top_features_dataframe_hippo, x="feature", y="value", hue="diagnosis", palette="pastel",
                    dodge=True, size=5, edgecolor="white", linewidth=1, legend=False)
    plt.savefig('out/top_feature_values_by_region.png', dpi=300, bbox_inches='tight')
    plt.clf()

In [ ]:
# plotting the values of all the important features


# with plt.xkcd():
#     sns.catplot(data=important_features_dataframe, kind="box", x="feature", y="value", hue="diagnosis", height=10, aspect=10, palette="pastel")        
#     plt.xticks(rotation=30)
#     plt.yscale('log')
#     plt.xlabel('Feature')
#     plt.ylabel('Size')
#     plt.title('Top Features', size=20)
#     plt.savefig('out/important_feature_values.png', dpi=300, bbox_inches='tight')
#     plt.show()
#     plt.clf()